In [ ]:
#%pip install polars pyarrow mlxtend
import polars as pl

import os
os.environ["POLARS_MAX_THREADS"] = "8"

In [ ]:
RAW = 'data/raw'
PROCESSED = 'data/processed'

In [21]:
df_users = pl.read_csv(f'{PROCESSED}/yelp_filtered_user_ids.csv', n_rows=500)

In [22]:
df_users

user_id
str
"""kCKcQAOM3lQQCjxQUySjfQ"""
"""Pt0_xRB8PQEzSjC43C-c-Q"""
"""rQd5YGWdO_DoUXXDkVlW9A"""
"""mri7UefuIlJtPvFDzqvMAg"""
"""KGfvWaSHWsQMNlzv02LB5A"""
…
"""K3W3CEwgrmjZYoODhvOE0w"""
"""hrxRk1zSk54qpkIugxyorQ"""
"""rZEIo6fjhbD0ZH5iF-uwpw"""


In [39]:
df_biz = pl.read_ndjson(f'{RAW}/business.json', n_rows=500)

In [40]:
df_biz.shape

(500, 14)

In [ ]:
biz = (
    pl.read_ndjson(f"{RAW}/business.json")
      .select("business_id", "categories")
      .with_columns(pl.col("categories").str.split(",").alias("cats"))
      .explode("cats")
      .with_columns(pl.col("cats").str.strip_chars().alias("cat"))  
      .filter(pl.col("cat").is_not_null() & (pl.col("cat") != ""))
      .select("business_id", "cat")
      .unique()
)


In [43]:
# select random id 
sample_biz_id = biz.select(pl.col("business_id")).sample(n=1).item()
print(f"Sample business_id: {sample_biz_id}")

Sample business_id: RmJJ979n9a2AZLUSe7_X6Q


In [ ]:
# display the following biz item RmJJ979n9a2AZLUSe7_X6Q
biz.filter(pl.col("business_id") == sample_biz_id)

business_id,cat
str,str
"""RmJJ979n9a2AZLUSe7_X6Q""","""Peruvian"""
"""RmJJ979n9a2AZLUSe7_X6Q""","""Latin American"""
"""RmJJ979n9a2AZLUSe7_X6Q""","""American (Traditional)"""
"""RmJJ979n9a2AZLUSe7_X6Q""","""Colombian"""
"""RmJJ979n9a2AZLUSe7_X6Q""","""Restaurants"""


In [ ]:
# load reviews 
reviews = pl.read_ndjson(f"{RAW}/review_clean.ndjson").select("review_id","user_id","business_id", "stars")